In [1]:
import ANNDataPrep
import tensorflow as tf
# import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.model_selection import KFold


In [4]:
import numpy as np
input_sequence_length = 12
forecast_horizon = 12     # 24 = 2 hr,    12 = 1 hr,     6 = 30 min
features = 13
path = "D:\MyData/Extracted Data"
X, y = ANNDataPrep.preprocessData(input_sequence_length, forecast_horizon, features, path)
# np.savetxt("X.csv", X, delimiter=",")
# np.savetxt("y.csv", y, delimiter=",")

--- 230.3761019706726 seconds ---
(373925, 12, 13)
(373925, 13)


NameError: name 'numpy' is not defined

In [6]:
import numpy as np
# np.savetxt("X.csv", X, delimiter=",")
# np.savetxt("y.csv", y, delimiter=",")
y = y[:,5:7]

ValueError: Expected 1D or 2D array, got 3D array instead

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

print("X_train size: " + str(X_train.shape))
print("y_train size: " + str(y_train.shape))
print("X_val size: " + str(X_val.shape))
print("y_val size: " + str(y_val.shape))
print("X_test size: " + str(X_test.shape))
print("y_test size: " + str(y_test.shape))
# numpy.savetxt("X_train.csv", X_train, delimiter=",")
# numpy.savetxt("y_train.csv", y_train, delimiter=",")
# numpy.savetxt("X_val.csv", X_val, delimiter=",")
# numpy.savetxt("y_val.csv", y_val, delimiter=",")
# numpy.savetxt("X_test.csv", X_test, delimiter=",")
# numpy.savetxt("y_test.csv", y_test, delimiter=",")

## Determine what params to predict

In [5]:
# y_train = y_train[:,5:7]
# y_val = y_val[:,5:7]
# y_test = y_test[:,5:7]

In [6]:
try:
    num_predicted = y_train.shape[1]
except TypeError:
    num_predicted = 1
# print(num_predicted)

## Create Model and train

In [7]:
# # https://stackoverflow.com/questions/61931629/overfitting-in-lstm-even-after-using-regularizers
# # def create_model():
# model = tf.keras.models.Sequential()
# # model.add(tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.15, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.30, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.30, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(tf.keras.layers.LSTM(16, return_sequences=True, dropout=0.30, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(tf.keras.layers.LSTM(8, return_sequences=False, dropout=0.30, input_shape=(X_train.shape[1], X_train.shape[2])))
# # model.add(tf.keras.layers.Dense(64, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.1)))
# # model.add(tf.keras.layers.Dense(64, activation='tanh'))
# model.add(tf.keras.layers.Dense(32, activation='linear'))
# model.add(tf.keras.layers.Dense(num_predicted, activation='linear'))
# # model.compile(optimizer='adam', loss='mse', metrics=['mse'])
# model.compile(optimizer='adam', loss='mse')
# model.summary()
#     # return model

In [8]:
# start_time = time.time()
# batch_size = 128
# model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=batch_size, verbose=1)
# print("--- %s seconds ---" % (time.time() - start_time))

# Model I am Using
### With 5 Fold Cross Validation

In [9]:
# https://stackoverflow.com/questions/61931629/overfitting-in-lstm-even-after-using-regularizers
def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.15, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(tf.keras.layers.Dense(num_predicted, activation='linear'))
    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    model.summary()
    return model
# start_time = time.time()
batch_size = 128
# model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val), batch_size=batch_size, verbose=1)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
for train_index, val_index in kf.split(X):
    X_train_new, X_val = X[train_index], X[val_index]
    y_train_new, y_val = y[train_index], y[val_index]
    
    model = create_model()
    model.fit(X_train_new, y_train_new, epochs=50, batch_size=batch_size, verbose=1)
    val_loss = model.evaluate(X_val, y_val, verbose=0)
    print(f'Fold {fold_no} - Validation Loss: {val_loss}')
    fold_no += 1

C:\Users\jared\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          72,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 72,962 (285.01 KB)

 Trainable params: 72,962 (285.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 83s 34ms/step - loss: 0.6054 - mse: 0.6571
Epoch 2/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 84s 34ms/step - loss: 0.5142 - mse: 0.5173
Epoch 3/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 81s 34ms/step - loss: 0.4915 - mse: 0.4831
Epoch 4/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 82s 34ms/step - loss: 0.4808 - mse: 0.4661
Epoch 5/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 84s 34ms/step - loss: 0.4720 - mse: 0.4520
Epoch 6/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 81s 34ms/step - loss: 0.4650 - mse: 0.4415
Epoch 7/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 80s 33ms/step - loss: 0.4605 - mse: 0.4344
Epoch 8/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 85s 34ms/step - loss: 0.4556 - mse: 0.4268
Epoch 9/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 79s 33ms/step - loss: 0.4524 - mse: 0.4216
Epoch 10/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 77s 33ms/step - loss: 0.4504 - mse: 0.4191
Epoch 11/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 73s 31ms/step - loss: 0.4464 - mse: 0.4124
Epoch 12/50
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 87s 34ms/step - loss

## Evaluate Results

In [ ]:
# trainScore = model.evaluate(X_train, y_train, verbose=0)
# print(trainScore)
# testScore = model.evaluate(X_test, y_test, verbose=0)
# print(testScore)


In [ ]:
# # https://stackoverflow.com/questions/61931629/overfitting-in-lstm-even-after-using-regularizers
# def create_model():
#     model = tf.keras.models.Sequential()
#     model.add(tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.15, input_shape=(X_train.shape[1], X_train.shape[2])))
#     model.add(tf.keras.layers.Dense(num_predicted, activation='linear'))
#     model.compile(optimizer='adam', loss='huber', metrics=['mse'])
#     model.summary()
# return model
# start_time = time.time()
# batch_size = 64
# model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val), batch_size=batch_size, verbose=1)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# trainScore = model.evaluate(X_train, y_train, verbose=0)
# print(trainScore)
# testScore = model.evaluate(X_test, y_test, verbose=0)
# print(testScore)
